In [ ]:
# @title 1. Install Dependencies
!pip install -q -U bitsandbytes
!pip install -q -U transformers accelerate
!pip install -q -U imagehash sentence-transformers

print("Dependencies installed.")

In [ ]:
# @title 2. Loading Model and Defining Logic Engine
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration, pipeline, BitsAndBytesConfig
from sentence_transformers import CrossEncoder
from PIL import Image, ImageChops, ImageEnhance
import spacy
import io
import imagehash
import base64
import warnings

warnings.filterwarnings("ignore")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

class NewsVerifier:
    def __init__(self):
        print("Loading Forensics & Vision Models...")
        self.fake_detector = pipeline("image-classification", model="umm-maybe/AI-image-detector", device=0 if device == "cuda" else -1)

        self.processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")

        try:
            print("   ...Attempting 4-bit quantization load...")
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16,
                bnb_4bit_quant_type="nf4"
            )
            self.model = Blip2ForConditionalGeneration.from_pretrained(
                "Salesforce/blip2-opt-2.7b",
                quantization_config=bnb_config,
                device_map="auto"
            )
            print("SUCCESS: 4-Bit Quantization Enabled!")
        except Exception as e:
            print(f"4-BIT FAILED with error: {e}")
            print("   ...Falling back to standard float16 loading.")
            self.model = Blip2ForConditionalGeneration.from_pretrained(
                "Salesforce/blip2-opt-2.7b",
                torch_dtype=torch.float16,
                device_map="auto"
            )

        print("Loading Logic/NLI Model...")
        self.nli_model = CrossEncoder('cross-encoder/nli-distilroberta-base', device=0)

        print("Loading NLP...")
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except OSError:
            from spacy.cli import download
            download("en_core_web_sm")
            self.nlp = spacy.load("en_core_web_sm")

        print("Systems Online.")
        self.known_fakes_db = {
            "d4d4d4d4d4d4d4d4": "Stock Photo: 'War Zone' (2015)",
            "a1a1a1a1a1a1a1a1": "Viral Hoax: 'Shark on Highway' (2018)",
        }

    def perform_ela(self, image):
        temp_buffer = io.BytesIO()
        image.convert("RGB").save(temp_buffer, format="JPEG", quality=90)
        temp_buffer.seek(0)

        ela_image = ImageChops.difference(image.convert("RGB"), Image.open(temp_buffer))
        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema]) or 1
        scale = 255.0 / max_diff
        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

        img_byte_arr = io.BytesIO()
        ela_image.save(img_byte_arr, format="PNG")
        encoded_image = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
        return f"data:image/png;base64,{encoded_image}"

    def detect_deepfake(self, image):
        try:
            results = self.fake_detector(image)
            for r in results:
                if r['label'] == 'artificial':
                    return round(r['score'] * 100, 2)
            return 0.0
        except: return 0.0

    def extract_claims(self, text):
        doc = self.nlp(text)
        entities = {"GPE": [], "DATE": [], "ORG": []}
        for ent in doc.ents:
            if ent.label_ in entities:
                entities[ent.label_].append(ent.text)
        return entities

    def ask_vqa(self, image, question):
        prompt = f"Question: {question} Answer:"
        inputs = self.processor(image, text=prompt, return_tensors="pt").to(device, torch.float16 if device == "cuda" else torch.float32)

        generated_ids = self.model.generate(**inputs, max_new_tokens=40, min_length=2, do_sample=False)
        output = self.processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
        return output.split("Answer:")[-1].strip() if "Answer:" in output else output

    def verify(self, image_bytes, text_caption):
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        flags = []

        # hash check
        img_hash = str(imagehash.phash(image))
        if img_hash in self.known_fakes_db:
            flags.append({"type": "Recycled Media", "severity": "Critical", "details": f"Known fake: {self.known_fakes_db[img_hash]}"})

        ai_prob = self.detect_deepfake(image)
        if ai_prob > 70:
            flags.append({"type": "AI Generated", "severity": "Critical", "details": f"{ai_prob}% probability of being AI."})


        scene_desc = self.ask_vqa(image, "Describe the environment, weather, and specific city or landmarks if known.")
        text_clues = self.ask_vqa(image, "What language or alphabet is visible on the signs?")
        objects = self.ask_vqa(image, "Describe the clothing, flags, and objects held by the people.")

        visual_summary = f"Scene: {scene_desc}. Visual Text: {text_clues}. Details: {objects}."
        print(f"DEBUG: Fact Sheet = {visual_summary}")

        # nli check
        scores = self.nli_model.predict([(text_caption, visual_summary)])[0]
        contradiction_score = scores[0]
        label_mapping = ['contradiction', 'entailment', 'neutral']
        predicted_label = label_mapping[scores.argmax()]
        if predicted_label == 'contradiction' and contradiction_score > 0.2:
            flags.append({
                "type": "Logical Contradiction",
                "severity": "Critical",
                "details": f"Visual evidence ({visual_summary}) contradicts the caption."
            })


        caption_claims = self.extract_claims(text_caption)
        summary_claims = self.extract_claims(visual_summary)

        caption_locs = [loc.lower() for loc in caption_claims.get("GPE", [])]
        summary_locs = [loc.lower() for loc in summary_claims.get("GPE", [])]

        if caption_locs:
            for cap_loc in caption_locs:

                if cap_loc not in visual_summary.lower():

                    conflicting_locs = [s_loc for s_loc in summary_locs if s_loc not in cap_loc]

                    if conflicting_locs:
                        flags.append({
                            "type": "Location Mismatch",
                            "severity": "Critical",
                            "details": f"Caption claims '{cap_loc}', but AI detected different location(s): {', '.join(conflicting_locs)}."
                        })

                    elif not summary_locs:
                        consistency_check = self.ask_vqa(image, f"Does this environment look like {cap_loc}? Answer yes or no.")
                        if "no" in consistency_check.lower():
                             flags.append({
                                "type": "Unverified Location",
                                "severity": "High",
                                "details": f"Visuals do not confirm the location is {cap_loc}."
                            })

        return {
            "ai_generated_probability": ai_prob,
            "ai_generated_caption": visual_summary,
            "detected_entities": caption_claims,
            "inconsistencies": flags,
            "raw_vqa_checks": visual_summary,
            "ela_heatmap": self.perform_ela(image)
        }

verifier_engine = NewsVerifier()

In [ ]:
# @title 3. Setup FastAPI & Dashboard

from fastapi import FastAPI, File, UploadFile, Form, Request
from fastapi.responses import HTMLResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
import uvicorn
import nest_asyncio
import os
import json

app = FastAPI()

# creating temp dir

os.makedirs("templates", exist_ok=True)
templates = Jinja2Templates(directory="templates")




# ~~ API ENDPOINTS

@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/analyze")
async def analyze(file: UploadFile = File(...), caption: str = Form(...)):
    contents = await file.read()

    result = verifier_engine.verify(contents, caption)
    return result

print("Server and Dashboard code ready.")

In [ ]:
!pip install pyngrok

In [ ]:
# @title 4. Start Server
import threading
from pyngrok import ngrok
import uvicorn
import nest_asyncio

ngrok.kill()

NGROK_TOKEN = "36ZUjCBnDA8k88Fjdo9AIcZy4Kf_3NKwPD9Ynebd3Ff3tpqjW"
ngrok.set_auth_token(NGROK_TOKEN)

try:
  public_url = ngrok.connect(8000).public_url
  print(f"\n Public URL: {public_url}\nClick the link above to open the dashboard.")
except Exception as e:
  print(f"Error starting tunnel: {e}")

def run_server():
  nest_asyncio.apply()
  uvicorn.run(app, port=8000, host="127.0.0.1", log_level="info")

thread = threading.Thread(target=run_server)
thread.start()